In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
import numpy as np
import jack
from lmz import *
import scanpy as sc

# LOAD DATA 

In [ ]:
'''
zedata = [sc.read('/home/ubuntu/benchdata/'+data) for data in "Immune_ALL_hum_mou.h5ad  Immune_ALL_human.h5ad  Lung_atlas_public.h5ad  human_pancreas_norm_complexBatch.h5ad".split()]
batch,typ = Transpose (Map(lambda x:x.split(), 'batch final_annotation#batch final_annotation#batch cell_type#tech celltype'.split("#")))
zedata =  [[z[z.obs[b]==i] for i in z.obs[b].unique()] for z,b in zip(zedata, batch)]
def samplecopy(data, num):
    obs_indices = np.random.choice(data.n_obs, size=num, replace=True,) 
    r=  data[obs_indices].copy()
    r.obs_names_make_unique()
    return r
data = [[samplecopy(d,500) for d in z] for z in zedata]
'''


# i think this was the way:
zedata = [sc.read('/home/ubuntu/benchdata/'+data) for data in "Immune_ALL_hum_mou.h5ad  Immune_ALL_human.h5ad  Lung_atlas_public.h5ad  human_pancreas_norm_complexBatch.h5ad".split()]
batch,typ = Transpose (Map(lambda x:x.split(), 'batch final_annotation#batch final_annotation#batch cell_type#tech celltype'.split("#")))

zedata =  [[z[z.obs[b]==i] for i in z.obs[b].unique()] for z,b in zip(zedata, batch)]
def samplecopy(data,num, seed):
    np.random.seed(seed) 
    obs_indices = np.random.choice(data.n_obs, size=num, replace=True,) 
    r=  data[obs_indices].copy()
    r.obs_names_make_unique()
    return r

ssdata = [[samplecopy(i,750,1234)  for i in series]  for series in zedata]



# SINGLE RUNS 

In [ ]:
from cellsaw import merge as merg
import cellsaw as cs
from cellsaw.merge.diffusion import kernel

# integrate 
data = ssdata[0]
cs.preprocess.annotate_genescore2(data,selector = 'cell_ranger',  normrow = True, log = True)
me = merg.Merge(data[0],umaps=[],oldcut=False, pca = 40)
dist = kernel.linear_assignment_integrate(me.projections[1], showtime = True)

In [ ]:
%%time
import networkx as nx
import ubergauss.tools as ug
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sklearn

def fold_draw(redmeth = 'nx'):
    # labels 
    y = [ x   for a in me.data for x in a.obs['batch'] ]
    y = [ x   for a in me.data for x in a.obs['final_annotation'] ]
    y =  ug.labelsToIntList(y)[0]

    if redmeth == 'nx':
        # make TSNE 
        g = nx.from_scipy_sparse_array(dist)
        pos = nx.spectral_layout(g,dim =2)
        X  = np.vstack(pos.values())
        #X = TSNE(n_components=2).fit_transform(X)
        pos = {i:e for i,e in enumerate(X)}
        # draw
        nx.draw_networkx_nodes(g, pos=pos, node_color =  y, node_size = 4, cmap = 'tab20')
    if redmeth == 'umap':
        coo = kernel.distmatrixumap(me.projections[1],dist,components = 2)
        plt.scatter(coo[:,0],coo[:,1], c = y)

    if redmeth == 'spectral':
        # sklearn.manifold.SpectralEmbedding
        coo = kernel.distmatrixumap(me.projections[1],dist,components = 2)
        plt.scatter(coo[:,0],coo[:,1], c = y)
        
    if redmeth == 'smacof':
        coo = sklearn.manifold.smacof(dist.toarray(), n_jobs = 1,n_components = 2)
        #coo = kernel.distmatrixumap(me.projections[1],dist,components = 2)
        plt.scatter(coo[:,0],coo[:,1], c = y)
fold_draw('umap')

# LOOP OVER ALL THE INTERESTS 

In [ ]:
from sklearn.metrics import  silhouette_score
import ubergauss.tools as ug
from natto.out.quality import clust as nnpurity 
from cellsaw.merge.diffusion import kernel
from cellsaw import merge as merg
import cellsaw as cs
import mnnpy
import umap
import time
mnnpy.settings.normalization = "single"
# we need the methods to return a single numpy array... 
sc._settings.ScanpyConfig.n_jobs = 0
sc._settings.ScanpyConfig.normalization = "single"

def mnn(merge):
    mnn = sc.external.pp.mnn_correct(*merge.data, n_jobs = None)
    X = umap.UMAP(n_components = 10,n_jobs =1).fit_transform(mnn[0].X)
    return X , "MNN", None

def mymethod(merge):
    data = merge.projections[1]
    dist = kernel.linear_assignment_integrate(data )
    sim = kernel.distmatrixumap(data,dist,components = 10)
    return  sim, "lsa_integration",dist

methods = [mnn,mymethod]

def myscore(x):
    start = time.time()
    method, rownorm, item, pp, sgenes  = x
    batch_,label = batch[item], typ[item]
    data = [z.copy() for z in ssdata[item]]
    
    # HVG normalized/unnormalized ROWNORM?
    if pp == 'seurat_v3':
        # wants count data so we do this firstL
        cs.preprocess.annotate_genescore2(data,selector = pp, 
                                          normrow = False, log = False)
    else: 
        cs.preprocess.annotate_genescore2(data,selector = pp, normrow = True, log = True)
    if rownorm:
        [ sc.pp.normalize_total(d,1e4) for d in data]
    
    [sc.pp.log1p(d) for d in data]
    me = merg.Merge(data,umaps=[],oldcut=False, pca = 40, selectgenes = sgenes)
    X ,methname, di  = method(me)
    y = [ x   for a in me.data for x in a.obs[label] ]
    r =  {"method":methname,"dataset":item,"normrow":rownorm,'pp':pp,'time':time.time()-start} 
    
    sh = silhouette_score(X,y)
    pur = nnpurity(X,y)
    def copyupdate(a,b):
        a = dict(a)
        a.update(b)
        return a
    
    if False:
        data = me.projections[2]
        X2 = kernel.distmatrixumap(data,dist,components = 2)
        plt.scatter(X2[:,0],X2[:,1], y)
        plt.show()
        
    s= copyupdate(r,{'score':sh,'metric':'silhouette'})
    p= copyupdate(r,{'score':pur,'metric':'purity'})
    return s,p 


#myscore((mymethod,True,0,'cell_ranger'))

res_newscoring = ug.xmap(myscore,
                         [ (method,True,item,pp,2000) for method in methods
                                                      for item in Range(zedata)
                                                      for pp in ['cell_ranger']]+
                         [ (mymethod,True,item,'cell_ranger',3000) 
                                                      for item in Range(zedata)])



In [ ]:
myscore((mymethod,True,1,'cell_ranger'))
myscore((mymethod,True,2,'cell_ranger'))
myscore((mymethod,True,3,'cell_ranger'))

# MY BOXPLOTS 

In [ ]:
for d in [a for aa in res_newscoring for a in aa][16:]:
    d['method'] = "lsa3000"

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
df2 = pd.DataFrame([a for aa in res_newscoring for a in aa])

p = df2[df2.metric == 'purity']
s = df2[df2.metric == 'silhouette']

sns.boxplot(data = p, y= 'score', x='method').set(title='acc')
plt.show()
sns.boxplot(data = s, y= 'score', x='method').set(title='silhou')
plt.show()
# so we do log1p and 


In [ ]:
from cellsaw import merge as merg
from cellsaw import preprocess as pp 
pp.annotate_genescore(zz,selector = 'cell_ranger')
me = merg.Merge(zz,umaps=[])
zz=me.data 

In [ ]:
mnn = sc.external.pp.mnn_correct(*zz, n_jobs = 30)

# comparison plots  (also f3 from mnn paper)

In [ ]:
from cellsaw import preprocess as pp 
from cellsaw import merge as merg
from cellsaw import draw 
from cellsaw import util
import umap
from sklearn.decomposition import PCA

def pcaplot(adata):
    d2 = PCA().fit_transform(adata.X) 
    d2split =  [d2[adata.obs['batch']==i].copy() for i in adata.obs['batch'].unique()]
    labels = [a.obs['celltype'] for a in me.data]
    
    def scatter(i,m):
        x = d2split[i]
        y = labels[i]
        plt.scatter(x[:,0],x[:,1],c=y,marker = m)
    scatter(0,"o")
    scatter(1,"x")
    plt.show()

pcaplot(mnn[0])
# combat, uncorrected, ours (mnn also)

In [ ]:
from cellsaw import preprocess as pp 
from cellsaw import merge as merg
from cellsaw import draw 
from cellsaw import util
import umap

from natto.out.quality import clust as nnpurity 
def sad():
    d2 = umap.UMAP().fit_transform(mnn[0].X) 
    d2split =  [d2[mnn[0].obs['batch']==i].copy() for i in mnn[0].obs['batch'].unique()]
    me.d2 = d2split
    labels = [a.obs['celltype'] for a in me.data]
    #me.plot(labels)
    mixlabels=[a for aa in labels for a in aa]
    me.plot(labels, mkmix= True, mixlabels = mixlabels)
    print("ACC",nnpurity(mnn[0].X,mixlabels))
sad()

In [ ]:


from cellsaw.merge.diffusion import kernel
data = me.projections[1]
dist = kernel.linear_assignment_integrate(data,intra_neigh = 10, inter_neigh = 20, scaling_num_neighbors = 3)


In [ ]:
def mds():
    d2 = kernel.distmatrixumap(data,dist)
    d2split = np.split(d2,np.add.accumulate(Map(len,data)))
    me.d2 = d2split
    labels = [a.obs['celltype'] for a in me.data]
    mixlabels=[a for aa in labels for a in aa]
    me.plot(labels, mkmix= True, mixlabels = mixlabels)
    print("ACC",nnpurity(d2,mixlabels))
    
mds()

# PLAY AROUND: MDS, UMAP CUSTOM GRAPHS, SILOUETTE SCORE

In [ ]:
from sklearn.datasets import make_blobs
import seaborn as sns

def plot(mydata):
    for d in mydata:
        plt.scatter(d[:,0],d[:,1])
    plt.show()
    
mydata = [make_blobs(n_samples = 50)[0] for i in range(2)]
plot(mydata)


dist = kernel.linear_assignment_integrate(mydata, intraneigh=3)
sns.heatmap(dist)
plt.show()

d2 = MDS(dissimilarity = 'precomputed',metric = False).fit_transform(dist)
d2 = np.split(d2,np.add.accumulate(Map(len,mydata)))
plot(d2)

d2 = kernel.distmatrixumap(mydata,dist)
d2 = np.split(d2,np.add.accumulate(Map(len,mydata)))
plot(d2)


# TODO
# networkx from numpoy array
# kamada_kawai_layout(G, dist=None, pos=None, weight='weight', scale=1, center=None, dim=2)
# also try spectral layout 

In [ ]:
from pynndescent import NNDescent
pyNNDobject = NNDescent(np.array([[1],[2],[3]]), metric='euclidean', random_state=1337)
pyNNDobject._neighbor_graph

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
import seaborn as sns

def plot(X,y):
    plt.scatter(X[:,0],X[:,1], c = y)
    plt.show()
    
X,y  = make_blobs(n_samples = 100, shuffle=False,centers = [(1,x*10) for x in range(2)])

print(silhouette_score(X,y))
plot(X,y)
X[:25,1]+=20
print(silhouette_score(X,y))
plot(X,y)